# TransformerEncoderLayer+TransformerDecoderLayer

https://www.jianshu.com/p/3ee76c01236b

## import Packages

In [46]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import pathlib
from pprint import pprint

#img
import cv2

#pytorch
import torch
from torch import nn
from torch import functional as F
from torch import optim
from torch.utils.data import Dataset,DataLoader,random_split
from torchvision import transforms
from torch.nn import Module
from torchvision import models
from PIL import Image


## 1. torch.nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')
TransformerEncoderLayer is made up of self-attn and feedforward network. This standard encoder layer is based on the paper “Attention Is All You Need”. Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in Neural Information Processing Systems, pages 6000-6010. Users may modify or implement in a different way during application.

Parameters:
>d_model – the number of expected features in the input (required).\
nhead – the number of heads in the multiheadattention models (required).\
dim_feedforward – the dimension of the feedforward network model (default=2048).\
dropout – the dropout value (default=0.1).\
activation – the activation function of intermediate layer, relu or gelu
(default=relu).

Examples:


In [47]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.rand(10, 32, 512)
out = encoder_layer(src)
print(out.size())

torch.Size([10, 32, 512])


forward(src, src_mask=None, src_key_padding_mask=None)

Pass the input through the encoder layer.

Parameters:
>src – the sequence to the encoder layer (required).\
src_mask – the mask for the src sequence (optional).\
src_key_padding_mask – the mask for the src keys per batch (optional).


### SOURCE CODE

In [48]:
class TransformerEncoderLayer(Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = Linear(d_model, dim_feedforward)
        self.dropout = Dropout(dropout)
        self.linear2 = Linear(dim_feedforward, d_model)

        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)

        self.activation = _get_activation_fn(activation)

    def __setstate__(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerEncoderLayer, self).__setstate__(state)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        # type: (Tensor, Optional[Tensor], Optional[Tensor]) -> Tensor

        src2 = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

## 2. torch.nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')

TransformerDecoderLayer is made up of self-attn, multi-head-attn and feedforward network. This standard decoder layer is based on the paper “Attention Is All You Need”. Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in Neural Information Processing Systems, pages 6000-6010. Users may modify or implement in a different way during application.

Parameters:
>d_model – the number of expected features in the input (required).\
nhead – the number of heads in the multiheadattention models (required).\
dim_feedforward – the dimension of the feedforward network model (default=2048).\
dropout – the dropout value (default=0.1).\
activation – the activation function of intermediate layer, relu or gelu (default=relu).

Examples:


In [49]:
decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
memory = torch.randn(10, 32, 512)
tgt = torch.randn(20, 32, 512)
out = decoder_layer(tgt, memory)
print(out.size())

torch.Size([20, 32, 512])


forward(tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)
Pass the inputs (and mask) through the decoder layer.

Parameters:
>tgt – the sequence to the decoder layer (required).\
memory – the sequence from the last layer of the encoder (required).\
tgt_mask – the mask for the tgt sequence (optional).\
memory_mask – the mask for the memory sequence (optional).\
tgt_key_padding_mask – the mask for the tgt keys per batch (optional).\
memory_key_padding_mask – the mask for the memory keys per batch (optional).


SOURCE CODE

In [50]:
class TransformerDecoderLayer(Module):
    
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = Linear(d_model, dim_feedforward)
        self.dropout = Dropout(dropout)
        self.linear2 = Linear(dim_feedforward, d_model)

        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.norm3 = LayerNorm(d_model)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)
        self.dropout3 = Dropout(dropout)

        self.activation = _get_activation_fn(activation)

    def __setstate__(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerDecoderLayer, self).__setstate__(state)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):
        # type: (Tensor, Tensor, Optional[Tensor], Optional[Tensor], Optional[Tensor], Optional[Tensor]) -> Tensor
        tgt2 = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        tgt2 = self.multihead_attn(tgt, memory, memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt

    def _get_clones(module, N):
         return ModuleList([copy.deepcopy(module) for i in range(N)])

    def _get_activation_fn(activation):
        if activation == "relu":
            return F.relu
        elif activation == "gelu":
            return F.gelu

        raise RuntimeError("activation should be relu/gelu, not {}".format(activation))


# 3. torch.nn.TransformerEncoder(encoder_layer, num_layers, norm=None)
TransformerEncoder is a stack of N encoder layers

Paremeters:
>encoder_layer – an instance of the TransformerEncoderLayer() class (required).\
num_layers – the number of sub-encoder-layers in the encoder (required).\
norm – the layer normalization component (optional).

Examples:


In [51]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
src = torch.randn(10, 32, 512)
out = transformer_encoder(src)
print(out.size())

torch.Size([10, 32, 512])


forward(src, mask=None, src_key_padding_mask=None)
Pass the input through the encoder layers in turn.

Parameters:
>src – the sequence to the encoder (required).\
mask – the mask for the src sequence (optional).\
src_key_padding_mask – the mask for the src keys per batch (optional).


SOURCE CODE

In [52]:
class TransformerEncoder(Module):
   
    __constants__ = ['norm']

    def __init__(self, encoder_layer, num_layers, norm=None):
        super(TransformerEncoder, self).__init__()
        self.layers = _get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    def forward(self, src, mask=None, src_key_padding_mask=None):
        # type: (Tensor, Optional[Tensor], Optional[Tensor]) -> Tensor
        output = src

        for mod in self.layers:
            output = mod(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask)

        if self.norm is not None:
            output = self.norm(output)

        return output

# 4. torch.nn.TransformerDecoder(decoder_layer, num_layers, norm=None)
TransformerDecoder is a stack of N decoder layers

Parameters:
>decoder_layer – an instance of the TransformerDecoderLayer() class (required).\
num_layers – the number of sub-decoder-layers in the decoder (required).\
norm – the layer normalization component (optional).

Examples:


In [53]:
decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
memory = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
out = transformer_decoder(tgt, memory)
print(out.size())

torch.Size([20, 32, 512])


forward(tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)
Pass the inputs (and mask) through the decoder layer in turn.

Parameters:
>tgt – the sequence to the decoder (required).\
memory – the sequence from the last layer of the encoder (required).\
tgt_mask – the mask for the tgt sequence (optional).\
memory_mask – the mask for the memory sequence (optional).\
tgt_key_padding_mask – the mask for the tgt keys per batch (optional).\
memory_key_padding_mask – the mask for the memory keys per batch (optional).

SOURCE CODE

In [54]:
class TransformerDecoder(Module):
    
    __constants__ = ['norm']

    def __init__(self, decoder_layer, num_layers, norm=None):
        super(TransformerDecoder, self).__init__()
        self.layers = _get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    def forward(self, tgt, memory, tgt_mask=None,
                memory_mask=None, tgt_key_padding_mask=None,
                memory_key_padding_mask=None):
        # type: (Tensor, Tensor, Optional[Tensor], Optional[Tensor], Optional[Tensor], Optional[Tensor]) -> Tensor
        
        output = tgt

        for mod in self.layers:
            output = mod(output, memory, tgt_mask=tgt_mask,
                         memory_mask=memory_mask,
                         tgt_key_padding_mask=tgt_key_padding_mask,
                         memory_key_padding_mask=memory_key_padding_mask)

        if self.norm is not None:
            output = self.norm(output)

        return output


# 5. torch.nn.Transformer(d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1, activation='relu', custom_encoder=None, custom_decoder=None)
A transformer model. User is able to modify the attributes as needed. The architecture is based on the paper “Attention Is All You Need”. Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in Neural Information Processing Systems, pages 6000-6010. Users can build the BERT(https://arxiv.org/abs/1810.04805) 
model with corresponding parameters.

Parameters:
>d_model – the number of expected features in the encoder/decoder inputs (default=512).\
nhead – the number of heads in the multiheadattention models (default=8).\
num_encoder_layers – the number of sub-encoder-layers in the encoder (default=6).\
num_decoder_layers – the number of sub-decoder-layers in the decoder (default=6).\
dim_feedforward – the dimension of the feedforward network model (default=2048).\
dropout – the dropout value (default=0.1).\
activation – the activation function of encoder/decoder intermediate layer, relu or gelu (default=relu).\
custom_encoder – custom encoder (default=None).\
custom_decoder – custom decoder (default=None).

Examples:

In [55]:
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12)
src = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
out = transformer_model(src, tgt)
print(out.size())

torch.Size([20, 32, 512])


### Note: A full example to apply nn.Transformer module for the word language model is available in 
https://github.com/pytorch/examples/tree/master/word_language_model

forward(src, tgt, src_mask=None, tgt_mask=None, memory_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)
Take in and process masked source/target sequences.

Parameters:

>src – the sequence to the encoder (required).\
tgt – the sequence to the decoder (required).\
src_mask – the additive mask for the src sequence (optional).\
tgt_mask – the additive mask for the tgt sequence (optional).\
memory_mask – the additive mask for the encoder output (optional).\
src_key_padding_mask – the ByteTensor mask for src keys per batch (optional).\
tgt_key_padding_mask – the ByteTensor mask for tgt keys per batch (optional).\
memory_key_padding_mask – the ByteTensor mask for memory keys per batch (optional).


Shape:
### src: **(S, N, E)**.
### tgt: **(T, N, E)**.
### src_mask:**(S, S)**.
### tgt_mask: **(T, T)**.
### memory_mask: **(T, S)**.

### src_key_padding_mask: **(N, S)**.

### tgt_key_padding_mask: **(N, T)**.

### memory_key_padding_mask: **(N, S)**.

Note: [src/tgt/memory]_mask should be filled with float(‘-inf’) for the masked positions and float(0.0) else. These masks ensure that predictions for position i depend only on the unmasked positions j and are applied identically for each sequence in a batch. [src/tgt/memory]_key_padding_mask should be a ByteTensor where True values are positions that should be masked with float(‘-inf’) and False values will be unchanged. This mask ensures that no information will be taken from position i if it is masked, and has a separate mask for each sequence in a batch.

### output:**(T, N, E)**.

Note: Due to the multi-head attention architecture in the transformer model, the output sequence length of a transformer is same as the input sequence (i.e. target) length of the decode.

where **S** is the source sequence length, **T** is the target sequence length, **N** is the batch size, **E** is the feature number

Examples:

output = transformer_model(src, tgt, src_mask=src_mask, tgt_mask=tgt_mask)

### generate_square_subsequent_mask(sz)

Generate a square mask for the sequence. The masked positions are filled with float(‘-inf’). Unmasked positions are filled with float(0.0).

SOURCE CODE


In [ ]:

class Transformer(Module):
    def __init__(self, d_model=512, nhead=8, num_encoder_layers=6,
                 num_decoder_layers=6, dim_feedforward=2048, dropout=0.1,
                 activation="relu", custom_encoder=None, custom_decoder=None):
        super(Transformer, self).__init__()

        if custom_encoder is not None:
            self.encoder = custom_encoder
        else:
            encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, activation)
            encoder_norm = LayerNorm(d_model)
            self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)

        if custom_decoder is not None:
            self.decoder = custom_decoder
        else:
            decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout, activation)
            decoder_norm = LayerNorm(d_model)
            self.decoder = TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm)

        self._reset_parameters()

        self.d_model = d_model
        self.nhead = nhead

    def forward(self, src, tgt, src_mask=None, tgt_mask=None,
                memory_mask=None, src_key_padding_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):
        # type: (Tensor, Tensor, Optional[Tensor], Optional[Tensor], Optional[Tensor], Optional[Tensor], Optional[Tensor], Optional[Tensor]) -> Tensor  # noqa
        if src.size(1) != tgt.size(1):
            raise RuntimeError("the batch number of src and tgt must be equal")

        if src.size(2) != self.d_model or tgt.size(2) != self.d_model:
            raise RuntimeError("the feature number of src and tgt must be equal to d_model")

        memory = self.encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        output = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=memory_mask,
                              tgt_key_padding_mask=tgt_key_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask)
        return output

    def generate_square_subsequent_mask(self, sz):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
 Unmasked positions are filled with float(0.0).
 """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def _reset_parameters(self):
        r"""Initiate parameters in the transformer model."""

        for p in self.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

## ISSUES
The generate_square_subsequent_mask function in nn.Transformer can only generate square masks, but memory_mask requires the dimension **(T, S)**. I am wondering is there a built in function in transformer?? Thank you!


In [57]:
def _generate_subsequent_mask(tgt_sz, src_sz):
    mask = (torch.triu(torch.ones(src_sz, tgt_sz)) == 1).transpose(0, 1)
    print(mask)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

_generate_subsequent_mask(4, 5)


tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True, False]])


tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf]])


### Answer 1.

You don’t need to use memory_mask unless you want to prevent the decoder from attending some tokens in the input sequence, and the original Transformer didn’t use it in the first place because the decoder should be aware of the entire input sequence for any token in the output sequence. The same thing can be said to the input sequence (i.e., src_mask.)

In the PyTorch language, the original Transformer settings are src_mask=None and memory_mask=None, and for tgt_mask=generate_square_subsequent_mask(T).

Again, memory_mask is used only when you don’t want to let the decoder attend certain tokens in the input sequence. That is why the input shape is  (where T is output sequence length and S is input sequence length.)

### 2. How to add padding mask to nn.TransformerEncoder module? More



In [58]:
def _generate_key_padding_mask(include_length): # return (N,L)
    max_length = torch.max(include_length)
    mask = torch.stack([torch.arange(max_length)>=i for i in include_length])
    return mask
    
include_length = torch.tensor([6, 4, 3, 2])

_generate_key_padding_mask(include_length)

tensor([[False, False, False, False, False, False],
        [False, False, False, False,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False,  True,  True,  True,  True]])

### 3. How to turn a list of tensor to tensor?


In [41]:
def _generate_key_padding_mask(include_length): # return (N,L)
    max_length = torch.max(include_length)
    print([torch.arange(max_length)>=i for i in include_length])
    mask = torch.stack([torch.arange(max_length)>=i for i in include_length])
    return mask
    
include_length = torch.tensor([6, 4, 3, 2])
_generate_key_padding_mask(include_length)

[tensor([False, False, False, False, False, False]), tensor([False, False, False, False,  True,  True]), tensor([False, False, False,  True,  True,  True]), tensor([False, False,  True,  True,  True,  True])]


tensor([[False, False, False, False, False, False],
        [False, False, False, False,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False,  True,  True,  True,  True]])